<a href="https://colab.research.google.com/github/HyeonhoonLee/marbling/blob/master/sirloin_image_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#sirloin image download from PHOTOZOU_API

In [ ]:
import sys, os, re, time
import urllib.request as req
import urllib.parse as parse
import json, xml
# API의 URL 지정하기
PHOTOZOU_API = "https://photozou.jp/rest/search_public.json"
CACHE_DIR = "/content/drive/My Drive/image/cache"
# 포토주 API로 이미지 검색하기 --- (※1)
def search_photo(keyword, offset=0, limit=100):
    # API 쿼리 조합하기
    keyword_enc = parse.quote_plus(keyword)
    q = "keyword={0}&offset={1}&limit={2}".format(keyword_enc, offset, limit)
    url = PHOTOZOU_API + "?" + q
    # 캐시 전용 폴더 만들기
    if not os.path.exists(CACHE_DIR):
        os.makedirs(CACHE_DIR)
    cache = CACHE_DIR + "/" + re.sub(r'[^a-zA-Z0-9\%\#]+', '_', url)
    if os.path.exists(cache):
        return json.load(open(cache, "r", encoding="utf-8"))
    print("[API] " + url)
    req.urlretrieve(url, cache)
    time.sleep(1) # --- 1초 쉬기
    return json.load(open(cache, "r", encoding="utf-8"))
# 이미지 다운로드하기 --- (※2)
def download_thumb(info, save_dir):
    if not os.path.exists(save_dir): os.makedirs(save_dir)
    if info is None: return
    if not "photo" in info["info"]:
        print("[ERROR] broken info")
        return
    photolist = info["info"]["photo"]
    for photo in photolist:
        title = photo["photo_title"]
        photo_id = photo["photo_id"]
        url = photo["thumbnail_image_url"]
        path = save_dir + "/" + str(photo_id) + "_thumb.jpg"
        if os.path.exists(path): continue
        try:
            print("[download]", title, photo_id)
            req.urlretrieve(url, path)
            time.sleep(1) # --- 1초 쉬기
        except Exception as e:
            print("[ERROR] failed to downlaod url=", url)
# 모두 검색하고 다운로드하기 --- (※3)
def download_all(keyword, save_dir, maxphoto = 1000):
    offset = 0
    limit = 100
    while True:
        # API 호출
        info = search_photo(keyword, offset=offset, limit=limit)
        if info is None:
            print("[ERROR] no result"); return
        if (not "info" in info) or (not "photo_num" in info["info"]):
            print("[ERROR] broken data"); return
        photo_num = info["info"]["photo_num"]
        if photo_num == 0:
            print("photo_num = 0, offset=", offset)
            return
        # 사진 정보가 포함돼 있으면 다운받기
        print("*** download offset=", offset)
        download_thumb(info, save_dir)
        offset += limit
        if offset >= maxphoto: break
if __name__ == '__main__':
    # 모듈로 사용할 수 있게 설정
    download_all("サーロイン", "/content/drive/My Drive/image/sirloin", maxphoto = 25000) # --- (※4)

[API] https://photozou.jp/rest/search_public.json?keyword=%E3%82%B5%E3%83%BC%E3%83%AD%E3%82%A4%E3%83%B3&offset=0&limit=100
*** download offset= 0
[download] 肉の日限定サーロインステーキ丼08 264952439
[download] 肉の日限定サーロインステーキ丼07 264952438
[download] 肉の日限定サーロインステーキ丼06 264952437
[download] 肉の日限定サーロインステーキ丼05 264952436
[download] 肉の日限定サーロインステーキ丼04 264952435
[download] 肉の日限定サーロインステーキ丼03 264952434
[download] 肉の日限定サーロインステーキ丼02 264952433
[download] 肉の日限定サーロインステーキ丼01 264952432
[download] サーロインステーキ 264876617
[download] ステーキ　サーロイン 264805889
[download] 輸入牛ステーキ用(サーロイン)1枚132g税込み425円 263200980
[download] 20190809「熟成肉の炙りサーロイン」324円 262670704
[download] CABアンガス牛サーロインステーキ 261846789
[download] 20190507夕 261453056
[download] 20190503夕 261363707
[download] 20190429「牛サーロイン肉」 261298120
[download] 20190429夕 261291743
[download] 静岡カントリー浜岡コース&ホテル　国産牛サーロインステーキ 261000542
[download] (2)サーロインいちもん焼き 260876745
[download] (4)牛サーロインステーキ～ガーリックグレービーソース～ 260874624
[download] 3/14 あさくま＠南柏 260730213
[download] サーロインペッパーステーキ 260515304
[down

KeyboardInterrupt: ignored

규동 판정

In [ ]:
import gyudon_keras as gyudon
import sys, os
from PIL import Image
import numpy as np
# 명령줄에서 파일 이름 지정하기 --- (※1)
if len(sys.argv) <= 1:
    print("gyudon-checker.py (<파일 이름>)")
    quit()
image_size = 50
categories = [
    "일반 규동", "생강 규동", 
    "양파 규동", "치즈 규동"]
calories = [656, 658, 768, 836]
# 입력 이미지를 Numpy로 변환하기 --- (※2)
X = []
files = []
for fname in sys.argv[1:]:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((image_size, image_size))
    in_data = np.asarray(img)
    X.append(in_data)
    files.append(fname)
X = np.array(X)
# CNN 모델 구축하기 --- (※3)
model = gyudon.build_model(X.shape[1:])
model.load_weights("./image/gyudon-model.hdf5")
# 데이터 예측하기 --- (※4)
html = ""
pre = model.predict(X)
for i, p in enumerate(pre):
    y = p.argmax()
    print("+입력:", files[i])
    print("|규동 이름:", categories[y])
    print("|칼로리:", calories[y])
    html += """
        <h3>입력:{0}</h3>
        <div>
          <p><img src="{1}" width=300></p>
          <p>규동 이름:{2}</p>
          <p>칼로리 :{3}kcal</p>
        </div>
    """.format(os.path.basename(files[i]),
        files[i],
        categories[y],
        calories[y])
# 리포트 저장하기 --- (※5)
html = "<html><body style='text-align:center;'>" + \
    "<style> p { margin:0; padding:0; } </style>" + \
    html + "</body></html>"
with open("gyudon-result.html", "w") as f:
    f.write(html)